In [1]:
import time
from tests.tools.spliceai import add_spliceai_eval_columns
import importlib
import pandas as pd
rd = importlib.import_module("app.back-end.data")

rd.download_selected_database_for_eys_gene("lovd", "", override=False)
rd.download_selected_database_for_eys_gene("gnomad", "", override=False)

lovd_data = rd.parse_lovd()
gnomad_data = rd.parse_gnomad()

rd.set_lovd_dtypes(lovd_data)
rd.set_gnomad_dtypes(gnomad_data)

variants_on_genome = lovd_data["Variants_On_Genome"].copy()

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id', 'VariantOnGenome/DNA', 'VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
data = rd.merge_gnomad_lovd(lovd_data, gnomad_data)
first_100_rows = data.head(100).copy()
fasta_path = "hg38_chr6.fa"
start_time = time.time()
result_data_spliceai = add_spliceai_eval_columns(first_100_rows[51:52], fasta_path)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to add SpliceAI evaluation column: {elapsed_time:.2f} seconds")
result_data_spliceai

The file at ../app/back-end/src/workspace/lovd/lovd_data.txt already exists.
The file at ../app/back-end/src/workspace/gnomad/gnomad_data.csv already exists.
Time taken to add SpliceAI evaluation column: 12.91 seconds


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,...,variant_id_gnomad,Delta score (acceptor gain)_spliceai,Delta score (acceptor loss)_spliceai,Delta score (donor gain)_spliceai,Delta score (donor loss)_spliceai,Delta position (acceptor gain)_spliceai,Delta position (acceptor loss)_spliceai,Delta position (donor gain)_spliceai,Delta position (donor loss)_spliceai,Max_Delta_Score_spliceai
0,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720567-T-G,0.0,0.0,0.0,0.0,63720577,63720522,63720577,63720605,0.0


In [2]:
def extract_chr6_sequence(input_file, output_file):
    """
    Extracts the sequence for chromosome 6 from a FASTA file and writes it to another file.

    :param input_file: Path to the input FASTA file.
    :param output_file: Path to the output file where chr6 sequence will be written.
    """
    with open(input_file, 'r') as fasta_file:
        with open(output_file, 'w') as output_fasta:
            write_sequence = False
            for line in fasta_file:
                line = line.strip()
                if line.startswith('>'):  # Header line
                    if 'chr6' in line:
                        write_sequence = True
                        output_fasta.write(line + '\n')  # Write the header for chr6
                    else:
                        write_sequence = False
                elif write_sequence:
                    output_fasta.write(line + '\n')  # Write sequence lines for chr6

# Usage
input_fasta_path = 'hg38.fa'  # Path to the input FASTA file
output_fasta_path = 'hg38_chr6.fa'  # Path to the output file for chr6

extract_chr6_sequence(input_fasta_path, output_fasta_path)
print(f"Extracted chr6 sequence and saved to {output_fasta_path}.")


Extracted chr6 sequence and saved to hg38_chr6.fa.
